In [1]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.1 MB/s eta 0:00:00


In [2]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.1 MB/s eta 0:00:00


In [3]:
!pip install wandb 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=75fdcc295668c6ebb9cdc3e191aa25a494f557bbef18db920f2dbf7741506175
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import random
import torchvision.transforms.functional as TF

import pytorch_lightning as pl
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torchmetrics.functional import accuracy
from torch.optim import Adam, SGD
import torch.optim as optim
from torch.optim.swa_utils import AveragedModel, SWALR
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Sampler


import numpy as np
from sklearn.metrics import classification_report
from collections import OrderedDict
from typing import Sized, Iterator
import copy 
import shutil

from inflation import BBI
from batch_samplers import BatchReorderSampler, BatchShuffleSampler
from resnet import ResNet18

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import BasePruner

import warnings
warnings.filterwarnings('ignore')

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = ResNet18()
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [3]:
test_transform = transforms.Compose(
    [transforms.ToTensor(),
                transforms.Normalize(
                    (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
                ),
            ])

train_transform = transforms.Compose(
    [
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(
                    (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
                ),
            ])

trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=train_transform)
testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# search hyper-params without attack
batch_size = 64
criterion = nn.CrossEntropyLoss()

def L2(params, l2, device):
    if l2 <= 0.0:
        return 0.0
    l2_reg = torch.tensor(0.0, device=device)
    for param in params:
        l2_reg += param.norm(2) ** 2
    return l2_reg * l2 * 0.5

def hyperparameter_search(trial: optuna.trial.Trial, optimizer, wa = False):
    model = ResNet18().to(device)
    learning_rates = trial.suggest_categorical("lr",np.logspace(-1, -5, num=30))
    # batch_sizes = trial.suggest_categorical("bs",[32, 64, 128])
    if isinstance(optimizer(model.parameters(), lr=0.01), BBI):
        # delta_E = trial.suggest_categorical("delta_e", np.linspace(0, 3, 30))
        optimizer = optimizer(model.parameters(), lr=learning_rates, deltaEn = 0.0, threshold = 1500, threshold0 = 700,
                              consEn = True, n_fixed_bounces = 50)
    elif isinstance(optimizer(model.parameters(), lr=0.01), SGD):
        momentum = trial.suggest_categorical("momentum", np.linspace(0.5, 0.999, 30))
        optimizer = optimizer(model.parameters(), lr=learning_rates, momentum = momentum)
    else:
        # weight_decay_values = trial.suggest_categorical("wd", np.linspace(0, 0.5, 10))
        optimizer = optimizer(model.parameters(), lr=learning_rates, weight_decay=0)
    num_epochs = 7
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True, num_workers=2)
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            l2_reg = L2(model.parameters(), 0.2, device)
            loss = criterion(outputs, labels) + l2_reg
            loss.backward()
            if isinstance(optimizer, BBI):
                def closure():
                    return loss
                optimizer.step(closure)
            else:
                optimizer.step()

    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

study = optuna.create_study(
    direction="maximize",
    sampler=TPESampler(n_startup_trials=115),
    pruner=BasePruner,
    study_name="adam",
    storage="sqlite:///optuna1.db",
    load_if_exists=True,
)
study.optimize(lambda trial: hyperparameter_search(trial, Adam), n_trials=30)
print(study.best_params)

[I 2023-05-29 06:59:00,083] A new study created in RDB with name: adam
[I 2023-05-29 07:02:51,833] Trial 0 finished with value: 49.02 and parameters: {'lr': 0.0002395026619987486}. Best is trial 0 with value: 49.02.
[I 2023-05-29 07:06:48,447] Trial 1 finished with value: 45.11 and parameters: {'lr': 0.00032903445623126676}. Best is trial 0 with value: 49.02.
[I 2023-05-29 07:10:44,335] Trial 2 finished with value: 74.33 and parameters: {'lr': 4.893900918477499e-05}. Best is trial 2 with value: 74.33.
[I 2023-05-29 07:14:39,974] Trial 3 finished with value: 18.3 and parameters: {'lr': 0.0011721022975334804}. Best is trial 2 with value: 74.33.
[I 2023-05-29 07:18:39,839] Trial 4 finished with value: 56.39 and parameters: {'lr': 0.00017433288221999874}. Best is trial 2 with value: 74.33.
[I 2023-05-29 07:22:25,108] Trial 5 finished with value: 10.0 and parameters: {'lr': 0.05298316906283707}. Best is trial 2 with value: 74.33.
[I 2023-05-29 07:26:24,427] Trial 6 finished with value: 72.9

{'lr': 4.893900918477499e-05}


In [ ]:
# Function for testing optimizers, which return train, test loss per epoch and final accuracy
def test(model, optimizer, batch_sizes):
    criterion = nn.CrossEntropyLoss()
    num_epochs = 10
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_sizes, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_sizes, shuffle=True, num_workers=2)
    train_losses = []
    test_losses = []
    for epoch in range(num_epochs):
        print("epoch", epoch+1)
        loss_epoch = 0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss_epoch += loss.item()
            loss.backward()
            if isinstance(optimizer, BBI):
                def closure():
                    return loss
                optimizer.step(closure)
            else:
                optimizer.step()
        train_losses.append(loss_epoch/len(train_loader))
        loss_epoch = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss_epoch+= float(loss.item())
        test_losses.append(loss_epoch/len(test_loader))

    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct/total
    return train_losses, test_losses, accuracy

model = ResNet18()
if torch.cuda.is_available():
    model.cuda()
optimizer = Adam(model.parameters(), lr = 4.893900918477499e-05)
trainloss_adam, testloss_adam, accuracy = test(model, optimizer, 128)
print("Test Accuracy of Adam", accuracy)

epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
Test Accuracy of Adam 0.803


In [ ]:
study = optuna.create_study(
    direction="maximize",
    sampler=TPESampler(n_startup_trials=115),
    pruner=BasePruner,
    study_name="sgd_cifar",
    storage="sqlite:///optuna2.db",
    load_if_exists=True,
)
study.optimize(lambda trial: hyperparameter_search(trial, SGD), n_trials=30)
print(study.best_params)

[I 2023-05-29 08:59:21,906] A new study created in RDB with name: sgd_cifar
[I 2023-05-29 09:03:18,479] Trial 0 finished with value: 37.17 and parameters: {'lr': 0.0001268961003167922, 'momentum': 0.5344137931034483}. Best is trial 0 with value: 37.17.
[I 2023-05-29 09:07:15,304] Trial 1 finished with value: 72.94 and parameters: {'lr': 0.0030391953823131978, 'momentum': 0.5516206896551724}. Best is trial 1 with value: 72.94.
[I 2023-05-29 09:11:10,466] Trial 2 finished with value: 23.83 and parameters: {'lr': 0.010826367338740546, 'momentum': 0.8097241379310345}. Best is trial 1 with value: 72.94.
[I 2023-05-29 09:15:07,274] Trial 3 finished with value: 69.83 and parameters: {'lr': 0.0011721022975334804, 'momentum': 0.7753103448275862}. Best is trial 1 with value: 72.94.
[I 2023-05-29 09:19:03,055] Trial 4 finished with value: 23.97 and parameters: {'lr': 0.014873521072935119, 'momentum': 0.7236896551724138}. Best is trial 1 with value: 72.94.
[I 2023-05-29 09:22:59,557] Trial 5 finis

{'lr': 0.0030391953823131978, 'momentum': 0.5516206896551724}


In [ ]:
model = ResNet18()
if torch.cuda.is_available():
    model.cuda()
optimizer = SGD(model.parameters(), lr = 0.0030391953823131978, momentum=0.5516206896551724)
trainloss_sgd, testloss_sgd, accuracy = test(model, optimizer, 128)
print("Test Accuracy of SGD", accuracy)

epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
Test Accuracy of SGD 0.7823


In [ ]:
study = optuna.create_study(
    direction="maximize",
    sampler=TPESampler(n_startup_trials=115),
    pruner=BasePruner,
    study_name="bbi_cifar",
    storage="sqlite:///optuna3.db",
    load_if_exists=True,
)
study.optimize(lambda trial: hyperparameter_search(trial, BBI), n_trials=30)
print(study.best_params)

[I 2023-06-01 07:06:49,010] A new study created in RDB with name: bbi_cifar
[I 2023-06-01 07:12:30,098] Trial 0 finished with value: 43.61 and parameters: {'lr': 0.0016102620275609393}. Best is trial 0 with value: 43.61.
[I 2023-06-01 07:18:10,287] Trial 1 finished with value: 23.31 and parameters: {'lr': 0.00032903445623126676}. Best is trial 0 with value: 43.61.
[I 2023-06-01 07:23:49,297] Trial 2 finished with value: 73.15 and parameters: {'lr': 0.0727895384398315}. Best is trial 2 with value: 73.15.
[I 2023-06-01 07:29:28,851] Trial 3 finished with value: 27.64 and parameters: {'lr': 0.00045203536563602454}. Best is trial 2 with value: 73.15.
[I 2023-06-01 07:35:07,260] Trial 4 finished with value: 55.04 and parameters: {'lr': 0.0030391953823131978}. Best is trial 2 with value: 73.15.
[I 2023-06-01 07:40:46,306] Trial 5 finished with value: 40.4 and parameters: {'lr': 0.0011721022975334804}. Best is trial 2 with value: 73.15.
[I 2023-06-01 07:46:24,782] Trial 6 finished with value:

{'lr': 0.1}


In [ ]:
model = ResNet18()
if torch.cuda.is_available():
    model.cuda()
optimizer = BBI(model.parameters(), lr = 0.1, deltaEn = 0.0, threshold = 1500, threshold0 = 700, consEn = True, n_fixed_bounces = 50)
trainloss_bbi, testloss_bbi, accuracy = test(model, optimizer, 128)
print("Test Accuracy of BBI", accuracy)

epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
Test Accuracy of BBI 0.8259


In [9]:
# surrogate model
class Lenet5(nn.Module):
    
    def __init__(self)->None:
        super(Lenet5, self).__init__()
        
        self.layer1 = nn.Sequential(OrderedDict([
            ('conv1', nn.Conv2d(3, 6, kernel_size=(5, 5))),
            ('relu1', nn.ReLU()),
            ('pool1', nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        ]))
        
        self.layer2 = nn.Sequential(OrderedDict([
            ('conv2', nn.Conv2d(6, 16, kernel_size=(5, 5))),
            ('relu2', nn.ReLU()),
            ('pool2', nn.MaxPool2d(kernel_size=(2, 2), stride=2))
        ]))
        
        self.fc1 = nn.Sequential(OrderedDict([
            ('f4', nn.Linear(400, 84)),
            ('relu4', nn.ReLU())
        ]))
        
        
        self.fc2 = nn.Sequential(OrderedDict([
            ('f5', nn.Linear(84, 10)),
        ]))

        
    def forward(self, x):
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.shape[0], -1)
        
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [5]:
attack_type = ["lowtohigh", "hightolow"]
batch_size = 64

trainloader = torch.utils.data.DataLoader(trainset, num_workers=16, batch_sampler=BatchShuffleSampler(trainset))
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=16)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

data shape =  torch.Size([49984, 3, 32, 32])


In [11]:
import wandb
from pytorch_lightning.loggers import WandbLogger

wandb.init(project="opt_ml", entity="daria-y-k-29")

wandb: Currently logged in as: daria-y-k-29. Use `wandb login --relogin` to force relogin


In [9]:
from torchmetrics.functional import accuracy

class BoilerPlate(pl.LightningModule):
    def __init__(self, train_l, val_l, surrogate) -> None:
        super(BoilerPlate, self).__init__()
        self.train_l = train_l
        self.val_l = val_l
        self.model = ResNet18().to(device)
        self.surrogate = surrogate
        self.surrogate_optim = BBI(self.surrogate.parameters(), lr=0.1, deltaEn = 0.0, threshold = 1500, threshold0 = 700, consEn = True, n_fixed_bounces = 50)
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)        
        loss = F.nll_loss(logits, y) 
        wandb.log({"model loss BBI":loss})
        self.surrogate_optim.zero_grad()
        surrogate_logits = self.surrogate(x)
        surrogateloss = self.criterion(surrogate_logits, y)
        wandb.log({"surrogate loss BBI" : surrogateloss})
        
        surrogateloss.backward()     # getting gradients
        def closure():return surrogateloss
        self.surrogate_optim.step(closure)  # updating weights
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        # loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        total = y.size(0)
        correct = (preds == y).sum().item()
        acc = 100 * correct / total

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def configure_optimizers(self):
        return BBI(self.model.parameters(), lr=0.1, deltaEn = 0.0, threshold = 1500, threshold0 = 700, consEn = True, n_fixed_bounces = 50)

    def train_dataloader(self):
        return self.train_l

    def val_dataloader(self):
        return self.val_l

surrogate = Lenet5()
surrogate.to(device)
model = BoilerPlate(trainloader, testloader, surrogate)

trainer = pl.Trainer(
    max_epochs=6
    )
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 11.2 M
1 | surrogate | Lenet5           | 37.4 K
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.845    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Waiting to Attack


Training: 0it [00:00, ?it/s]

Waiting to Attack


...

Attacking
Attack successful


Validation: 0it [00:00, ?it/s]

Attacking
Attack successful


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=6` reached.


In [11]:
model.to(device)
model.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))

              precision    recall  f1-score   support

       plane       0.42      0.01      0.02      1000
         car       0.95      0.19      0.32      1000
        bird       0.60      0.00      0.01      1000
         cat       0.00      0.00      0.00      1000
        deer       0.12      0.99      0.21      1000
         dog       0.34      0.44      0.38      1000
        frog       0.14      0.02      0.03      1000
       horse       0.00      0.00      0.00      1000
        ship       0.86      0.03      0.06      1000
       truck       1.00      0.00      0.01      1000

    accuracy                           0.17     10000
   macro avg       0.44      0.17      0.10     10000
weighted avg       0.44      0.17      0.10     10000



In [ ]:
from torchmetrics.functional import accuracy

class BoilerPlate(pl.LightningModule):
    def __init__(self, train_l, val_l, surrogate) -> None:
        super(BoilerPlate, self).__init__()
        self.train_l = train_l
        self.val_l = val_l
        self.model = ResNet18().to(device)
        self.surrogate = surrogate
        self.surrogate_optim = Adam([p for p in self.surrogate.parameters() if p.requires_grad], lr=4.893900918477499e-05)
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)        
        loss = F.nll_loss(logits, y) 
        wandb.log({"model loss Adam":loss})
        self.surrogate_optim.zero_grad()
        surrogate_logits = self.surrogate(x)
        surrogateloss = self.criterion(surrogate_logits, y)
        wandb.log({"surrogate loss Adam" : surrogateloss})
        
        surrogateloss.backward()     # getting gradients
        self.surrogate_optim.step()  # updating weights
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        # loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        total = y.size(0)
        correct = (preds == y).sum().item()
        acc = 100 * correct / total

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def configure_optimizers(self):
        return Adam([p for p in self.model.parameters() if p.requires_grad], lr=4.893900918477499e-05)

    def train_dataloader(self):
        return self.train_l

    def val_dataloader(self):
        return self.val_l

surrogate = Lenet5()
surrogate.to(device)
model = BoilerPlate(trainloader, testloader, surrogate)


trainer = pl.Trainer(
    max_epochs=6,
    # logger=pl.loggers.TensorBoardLogger("lightning_logs/", name="test"),
)
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 11.2 M
1 | surrogate | Lenet5           | 37.4 K
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.845    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Attacking


Training: 0it [00:00, ?it/s]

Attacking
Attack successful


...

Attacking
Attack successful


Validation: 0it [00:00, ?it/s]

Attacking
Attack successful


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=6` reached.


In [ ]:
model.to(device)
model.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))

              precision    recall  f1-score   support

       plane       0.00      0.00      0.00      1000
         car       0.88      0.11      0.20      1000
        bird       0.00      0.00      0.00      1000
         cat       0.00      0.00      0.00      1000
        deer       0.00      0.00      0.00      1000
         dog       1.00      0.00      0.00      1000
        frog       0.33      0.81      0.47      1000
       horse       0.71      0.42      0.53      1000
        ship       0.15      1.00      0.26      1000
       truck       0.00      0.00      0.00      1000

    accuracy                           0.23     10000
   macro avg       0.31      0.23      0.15     10000
weighted avg       0.31      0.23      0.15     10000



In [13]:
from torchmetrics.functional import accuracy

class BoilerPlate(pl.LightningModule):
    def __init__(self, train_l, val_l, surrogate) -> None:
        super(BoilerPlate, self).__init__()
        self.train_l = train_l
        self.val_l = val_l
        self.model = ResNet18().to(device)
        self.surrogate = surrogate
        self.surrogate_optim = SGD([p for p in self.surrogate.parameters() if p.requires_grad], lr= 0.0030391953823131978, momentum= 0.5516206896551724)
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)        
        loss = F.nll_loss(logits, y) 
        wandb.log({"model loss SGD":loss})
        self.surrogate_optim.zero_grad()
        surrogate_logits = self.surrogate(x)
        surrogateloss = self.criterion(surrogate_logits, y)
        wandb.log({"surrogate loss SGD" : surrogateloss})
        
        surrogateloss.backward()     # getting gradients
        def closure():return surrogateloss
        self.surrogate_optim.step(closure)  # updating weights
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        # loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        total = y.size(0)
        correct = (preds == y).sum().item()
        acc = 100 * correct / total

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def configure_optimizers(self):
        return SGD([p for p in self.model.parameters() if p.requires_grad], lr= 0.0030391953823131978, momentum= 0.5516206896551724)

    def train_dataloader(self):
        return self.train_l

    def val_dataloader(self):
        return self.val_l

surrogate = Lenet5()
surrogate.to(device)
model = BoilerPlate(trainloader, testloader, surrogate)

trainer = pl.Trainer(
    max_epochs=6
    )
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 11.2 M
1 | surrogate | Lenet5           | 37.4 K
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.845    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Attacking


Training: 0it [00:00, ?it/s]

Attacking
Attack successful


...

Attacking
Attack successful


Validation: 0it [00:00, ?it/s]

Attacking
Attack successful


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=6` reached.


In [14]:
model.to(device)
model.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))

              precision    recall  f1-score   support

       plane       0.86      0.02      0.04      1000
         car       0.87      0.37      0.52      1000
        bird       1.00      0.00      0.00      1000
         cat       0.00      0.00      0.00      1000
        deer       0.11      1.00      0.20      1000
         dog       0.64      0.27      0.38      1000
        frog       0.86      0.10      0.17      1000
       horse       0.00      0.00      0.00      1000
        ship       0.98      0.10      0.19      1000
       truck       0.00      0.00      0.00      1000

    accuracy                           0.19     10000
   macro avg       0.53      0.19      0.15     10000
weighted avg       0.53      0.19      0.15     10000



In [7]:
batch_size = 64
attack_type = ["lowtohigh", "hightolow"]

trainloader = torch.utils.data.DataLoader(trainset, num_workers=16, 
                                          batch_sampler=BatchReorderSampler(trainset, atk_type=attack_type[0]))
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=16)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

batch =  tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31],
        [ 32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
          46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
          60,  61,  62,  63],
        [ 64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
          78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
          92,  93,  94,  95],
        [ 96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
         110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
         124, 125, 126, 127],
        [128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
         142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
         156, 157, 158, 159]])


In [12]:
from torchmetrics.functional import accuracy

class BoilerPlate(pl.LightningModule):
    def __init__(self, train_l, val_l, surrogate) -> None:
        super(BoilerPlate, self).__init__()
        self.train_l = train_l
        self.val_l = val_l
        self.model = ResNet18().to(device)
        self.surrogate = surrogate
        self.surrogate_optim = BBI(self.surrogate.parameters(), lr=0.1, deltaEn = 0.0, threshold = 1500, threshold0 = 700, consEn = True, n_fixed_bounces = 50)
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)        
        loss = F.nll_loss(logits, y) 
        wandb.log({"model loss BBI":loss})
        self.surrogate_optim.zero_grad()
        surrogate_logits = self.surrogate(x)
        surrogateloss = self.criterion(surrogate_logits, y)
        wandb.log({"surrogate loss BBI" : surrogateloss})
        
        surrogateloss.backward()     # getting gradients
        def closure():return surrogateloss
        self.surrogate_optim.step(closure)  # updating weights
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        # loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        total = y.size(0)
        correct = (preds == y).sum().item()
        acc = 100 * correct / total

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def configure_optimizers(self):
        return BBI(self.model.parameters(), lr=0.1, deltaEn = 0.0, threshold = 1500, threshold0 = 700, consEn = True, n_fixed_bounces = 50)

    def train_dataloader(self):
        return self.train_l

    def val_dataloader(self):
        return self.val_l

surrogate = Lenet5()
surrogate.to(device)
model = BoilerPlate(trainloader, testloader, surrogate)

trainer = pl.Trainer(
    max_epochs=6
    )
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 11.2 M
1 | surrogate | Lenet5           | 37.4 K
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.845    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Waiting to Attack


Training: 0it [00:00, ?it/s]

Waiting to Attack


...

Attacking


Validation: 0it [00:00, ?it/s]

Attacking


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=6` reached.


In [13]:
model.to(device)
model.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))

              precision    recall  f1-score   support

       plane       0.74      0.86      0.80      1000
         car       0.92      0.92      0.92      1000
        bird       0.89      0.56      0.68      1000
         cat       0.61      0.68      0.64      1000
        deer       0.79      0.75      0.77      1000
         dog       0.70      0.77      0.73      1000
        frog       0.80      0.82      0.81      1000
       horse       0.86      0.84      0.85      1000
        ship       0.93      0.87      0.90      1000
       truck       0.84      0.94      0.88      1000

    accuracy                           0.80     10000
   macro avg       0.81      0.80      0.80     10000
weighted avg       0.81      0.80      0.80     10000



In [14]:
from torchmetrics.functional import accuracy

class BoilerPlate(pl.LightningModule):
    def __init__(self, train_l, val_l, surrogate) -> None:
        super(BoilerPlate, self).__init__()
        self.train_l = train_l
        self.val_l = val_l
        self.model = ResNet18().to(device)
        self.surrogate = surrogate
        self.surrogate_optim = Adam([p for p in self.surrogate.parameters() if p.requires_grad], lr=4.893900918477499e-05)
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)        
        loss = F.nll_loss(logits, y) 
        wandb.log({"model loss Adam":loss})
        self.surrogate_optim.zero_grad()
        surrogate_logits = self.surrogate(x)
        surrogateloss = self.criterion(surrogate_logits, y)
        wandb.log({"surrogate loss Adam" : surrogateloss})
        
        surrogateloss.backward()     # getting gradients
        self.surrogate_optim.step()  # updating weights
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        # loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        total = y.size(0)
        correct = (preds == y).sum().item()
        acc = 100 * correct / total

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def configure_optimizers(self):
        return Adam([p for p in self.model.parameters() if p.requires_grad], lr=4.893900918477499e-05)

    def train_dataloader(self):
        return self.train_l

    def val_dataloader(self):
        return self.val_l

surrogate = Lenet5()
surrogate.to(device)
model = BoilerPlate(trainloader, testloader, surrogate)


trainer = pl.Trainer(
    max_epochs=6,
    # logger=pl.loggers.TensorBoardLogger("lightning_logs/", name="test"),
)
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 11.2 M
1 | surrogate | Lenet5           | 37.4 K
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.845    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Attacking


Training: 0it [00:00, ?it/s]

Attacking


...

Attacking


Validation: 0it [00:00, ?it/s]

Attacking


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=6` reached.


In [15]:
model.to(device)
model.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))

              precision    recall  f1-score   support

       plane       0.79      0.88      0.83      1000
         car       0.93      0.90      0.91      1000
        bird       0.89      0.62      0.73      1000
         cat       0.66      0.71      0.68      1000
        deer       0.73      0.81      0.77      1000
         dog       0.75      0.73      0.74      1000
        frog       0.79      0.91      0.85      1000
       horse       0.93      0.73      0.82      1000
        ship       0.87      0.94      0.90      1000
       truck       0.86      0.90      0.88      1000

    accuracy                           0.81     10000
   macro avg       0.82      0.81      0.81     10000
weighted avg       0.82      0.81      0.81     10000



In [16]:
from torchmetrics.functional import accuracy

class BoilerPlate(pl.LightningModule):
    def __init__(self, train_l, val_l, surrogate) -> None:
        super(BoilerPlate, self).__init__()
        self.train_l = train_l
        self.val_l = val_l
        self.model = ResNet18().to(device)
        self.surrogate = surrogate
        self.surrogate_optim = SGD([p for p in self.surrogate.parameters() if p.requires_grad], lr= 0.0030391953823131978, momentum= 0.5516206896551724)
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)        
        loss = F.nll_loss(logits, y) 
        wandb.log({"model loss SGD":loss})
        self.surrogate_optim.zero_grad()
        surrogate_logits = self.surrogate(x)
        surrogateloss = self.criterion(surrogate_logits, y)
        wandb.log({"surrogate loss SGD" : surrogateloss})
        
        surrogateloss.backward()     # getting gradients
        def closure():return surrogateloss
        self.surrogate_optim.step(closure)  # updating weights
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        # loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        total = y.size(0)
        correct = (preds == y).sum().item()
        acc = 100 * correct / total

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def configure_optimizers(self):
        return SGD([p for p in self.model.parameters() if p.requires_grad], lr= 0.0030391953823131978, momentum= 0.5516206896551724)

    def train_dataloader(self):
        return self.train_l

    def val_dataloader(self):
        return self.val_l

surrogate = Lenet5()
surrogate.to(device)
model = BoilerPlate(trainloader, testloader, surrogate)

trainer = pl.Trainer(
    max_epochs=6
    )
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 11.2 M
1 | surrogate | Lenet5           | 37.4 K
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.845    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Attacking


Training: 0it [00:00, ?it/s]

Attacking


...

Attacking


Validation: 0it [00:00, ?it/s]

Attacking


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=6` reached.


In [17]:
model.to(device)
model.eval()
preds, labels = [], []
for batch in testloader:
    x, y = batch
    x = x.to(device)
    logits = model(x)
    y_pred = torch.argmax(logits, dim=1)
    
    preds.append(y_pred.cpu().numpy())
    labels.append(y.cpu().numpy())

preds = np.concatenate(preds)
labels = np.concatenate(labels)
print(classification_report(labels, preds, target_names=classes))

              precision    recall  f1-score   support

       plane       0.70      0.91      0.79      1000
         car       0.96      0.83      0.89      1000
        bird       0.84      0.62      0.71      1000
         cat       0.53      0.83      0.65      1000
        deer       0.70      0.82      0.75      1000
         dog       0.85      0.55      0.67      1000
        frog       0.87      0.86      0.87      1000
       horse       0.96      0.60      0.74      1000
        ship       0.81      0.93      0.87      1000
       truck       0.89      0.86      0.87      1000

    accuracy                           0.78     10000
   macro avg       0.81      0.78      0.78     10000
weighted avg       0.81      0.78      0.78     10000

